
# 1. Pourquoi tokeniser ? 


In [1]:


text = "Les modèles de langage sont puissants."

print("Texte brut :")
print(text)

print("\nRemarque :")
print("Les modèles ne comprennent PAS directement le texte.")
print("Ils doivent le transformer en une séquence d'entiers : x1, x2, ..., xT.")


Texte brut :
Les modèles de langage sont puissants.

Remarque :
Les modèles ne comprennent PAS directement le texte.
Ils doivent le transformer en une séquence d'entiers : x1, x2, ..., xT.


## Remarques:

1- Un LLM ne lit pas du texte, seulement des entiers.

2- La tokenisation est la première étape pour transformer du texte → tokens → IDs.

3- Différentes stratégies existent.

# Tokenisation naïve : mot-à-mot

In [3]:

text = "Les modèles de langage sont puissants."

tokens = text.split()

print("Tokenisation mot-à-mot :")
print(tokens)
print("\nNombre de tokens :", len(tokens))


Tokenisation mot-à-mot :
['Les', 'modèles', 'de', 'langage', 'sont', 'puissants.']

Nombre de tokens : 6


Ce qu’on observe :

- Trop simple.

- Sensible à la ponctuation ("puissants." ≠ "puissants").

- Impossible de gérer les fautes, variantes, néologismes.

- Vocabulaire gigantesque → pas viable pour les LLMs.

# 3. Limites de la tokenisation mot-à-mot

In [4]:
texts = [
    "robot",
    "robots",
    "robotique",
    "robotiques"
]

for t in texts:
    print(f"{t:12s} -> {t.split()}")


robot        -> ['robot']
robots       -> ['robots']
robotique    -> ['robotique']
robotiques   -> ['robotiques']


# Remarques:

Chaque forme est vue comme un mot différent → explosion du vocabulaire.

Aucun partage d’information entre “robot”, “robots”, “robotique”…

Dans une langue riche (français), c’est ingérable.

# Tokenisation caractère par caractère

In [6]:
text = "robotique"

chars = list(text)
print(chars)
print("Nombre de 'tokens' :", len(chars))

['r', 'o', 'b', 'o', 't', 'i', 'q', 'u', 'e']
Nombre de 'tokens' : 9


# Remarques:

- Résout le problème du vocabulaire.
- Mais rend la séquence trop longue.
- Le modèle doit “réapprendre” toutes les combinaisons possibles → inefficace.

# Idée clé du BPE (Byte-Pair Encoding)

In [7]:
print("Byte-Pair Encoding (BPE) :")
print("- On commence avec des caractères.")
print("- On fusionne les paires les plus fréquentes.")
print("- On répète jusqu'à obtenir un vocabulaire de sous-mots.")

Byte-Pair Encoding (BPE) :
- On commence avec des caractères.
- On fusionne les paires les plus fréquentes.
- On répète jusqu'à obtenir un vocabulaire de sous-mots.


BPE est un compromis :

- pas trop long (pas de caractères),
- pas trop gros (pas de mot complet),
- capable de gérer les mots inconnus → découpage en sous-unités.


# 6. Tokenisation BPE réelle (GPT-2)


In [8]:


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "internationalisation"
tokens = tokenizer.tokenize(text)

print("Texte :", text)
print("\nTokens BPE :", tokens)


Texte : internationalisation

Tokens BPE : ['international', 'isation']


Remarques:
    
- Le modèle découpe en sous-mots du corpus BPE GPT-2.
-  Le découpage peut être surprenant → dépend de la fréquence dans le corpus Web.

In [ ]:
# 7 Obtenir les IDs associés aux tokens

In [10]:
ids = tokenizer.encode(text)

print("Tokens :", tokens)
print("IDs    :", ids)
print("\nNombre de tokens BPE :", len(tokens))


Tokens : ['international', 'isation']
IDs    : [45609, 5612]

Nombre de tokens BPE : 2


In [ ]:
# 8. Comparer la tokenisation sur plusieurs phrases

In [11]:
examples = [
    "robot",
    "robotique",
    "robots industriels",
    "Les humanoïdes collaborent avec les humains."
]

for t in examples:
    print("\n=== Phrase :", t)
    print("Tokens :", tokenizer.tokenize(t))
    print("IDs    :", tokenizer.encode(t))



=== Phrase : robot
Tokens : ['ro', 'bot']
IDs    : [305, 13645]

=== Phrase : robotique
Tokens : ['ro', 'bot', 'ique']
IDs    : [305, 13645, 2350]

=== Phrase : robots industriels
Tokens : ['rob', 'ots', 'Ġindust', 'ri', 'els']
IDs    : [22609, 1747, 2226, 380, 1424]

=== Phrase : Les humanoïdes collaborent avec les humains.
Tokens : ['Les', 'Ġhuman', 'o', 'Ã¯', 'des', 'Ġcoll', 'ab', 'ore', 'nt', 'Ġa', 'vec', 'Ġles', 'Ġhum', 'ains', '.']
IDs    : [35882, 1692, 78, 26884, 8906, 2927, 397, 382, 429, 257, 35138, 10287, 1311, 1299, 13]


# Remarques

- Certains mots sont coupés (Ġrobots, industri, els)
- Les espaces sont codés avec un préfixe spécial (Ġ)
- Les mots inconnus sont décomposés en sous-unités → robustesse.